Notebook to commence creation of DataSet2 creation.

In [1]:
import pandas as pd
import numpy as np

In [2]:
ED = pd.read_pickle('./../../data/raw/sghED.pkl')

In [3]:
IP = pd.read_pickle('./../../data/raw/sghIPspell.pkl')

In [5]:
IP.head()

,hosp_patid,adm_datetime,move_total,adm_loc,dis_loc,admission_method,admission_type,spel_los,gender,site,...,dis_month,dis_week,dis_dayofweek_name,dis_year,dis_datetime,dis_destination,dis_method,dis_day,dis_date,dis_flag_wkend
0,90,2011-09-13 00:12:00,2,CLINICAL DECISIONS UNIT AE,2,21,Non-Elective,7,2,SGH,...,9.0,38.0,Tuesday,2011.0,2011-09-20 20:00:00,19.0,1.0,20.0,2011-09-20,0
1,90,2011-09-30 13:42:00,1,CLINICAL DECISIONS UNIT AE,1,21,Day Case,0,2,SGH,...,9.0,39.0,Friday,2011.0,2011-09-30 17:44:00,19.0,1.0,30.0,2011-09-30,0
2,90,2012-11-21 09:28:00,1,WARD F3,1,21,Non-Elective,8,2,SGH,...,11.0,48.0,Thursday,2012.0,2012-11-29 15:00:00,51.0,1.0,29.0,2012-11-29,0
3,90,2013-12-12 10:00:00,1,MEDICAL ENDOSCOPY UNIT E LEVEL,1,11,Day Case,0,2,SGH,...,12.0,50.0,Thursday,2013.0,2013-12-12 13:15:00,19.0,1.0,12.0,2013-12-12,0
4,90,2014-07-20 14:32:00,1,CLINICAL DECISIONS UNIT AE,1,21,Day Case,0,2,SGH,...,7.0,29.0,Sunday,2014.0,2014-07-20 16:45:00,19.0,1.0,20.0,2014-07-20,1


#### get data for target

In [10]:
target = pd.read_pickle('./../../data/interim/EDooc_class.pkl')
target.head()

,EDocc,flag_target
dt_date,,
2012-01-01,55.750000,1
2012-01-02,38.541667,0
2012-01-03,40.208333,0
2012-01-04,32.375000,0
2012-01-05,32.125000,0



# Make Small Set for quick dev

In [ ]:
IP.query()

# Functions to create daily features

## Day Instance calcs

ED: 
- no of records (no in dept)
-

IP:
- no of patients in beds
- age: mean, prop >65
- no/proportion admission type

In [ ]:
def select_records_at_time(df, datetime):
    return

df[(df[start_col] <= datetime_hour) & (df[end_col] > datetime_hour)]

## Summary Daily calcs

##### Vars to calc
ED: 
- no attendances
- no admissions
- no ambulances
- ages: mean, std, no > 65, no >75
- 

IP:
- no admissions
    (by type: elec, non-elec, etc)
    (by age: mean, > 65, > 75)
    
- no discharges

Extras (very complicated):
- peak occupancy of day
- % occupancy, wrt to total numbers of beds (from previous)

In [66]:
for date in target.index[0:2]:
    print(date)

2012-01-01 00:00:00
2012-01-02 00:00:00


In [79]:
def select_records_in_period(df, datetimes_start, datetime_end, start_col, end_col):
    """
    Takes df and selects only records within time period.
    
    Return
    ======
    sample, dataframe , with only records which are of interest.
    
    """
    sample = df[(df[start_col] < datetime_end) & (df[end_col] >= datetimes_start)]
    return(sample)

def ED_period_calcs(df_sample):
    """
    Make calculations for the period
    """
    calcs = {}
    calcs['attendances'] = len(df_sample) # no attendances
    calcs['admissions'] = len(df_sample[df_sample.adm_flag == 1])
    calcs['age_mean'] = df_sample['age'].mean()
    calcs['age_65plus'] = len(df_sample[df_sample.age >= 65])
    
    
    return(calcs)





def get_daily_df(df, datetimes, df_type):
    """
    Calculate daily features from patient level records.
    
    Input
    =====
    df, dateframe, patient level records.
    datetimes, DateTimeIndex, datetime index of each date features are required for. 
    
    Return
    ======
    
    
    """
    if df_type == 'ED':
        calculation_function = ED_period_calcs
    elif df_type == 'IP':
        print('MAKE_IP_FUNC!')
    
    #### setup new df for results
    daily_features = pd.DataFrame(index=datetimes, columns = ['attendances', 'admissions', 'age_mean', 'age_65plus'])
    
    #### loop over all records
    for start_date in datetimes:
        #### get records
#         print(start_date)
        end_date = start_date + pd.Timedelta(1, 'd')# cal enddate (1 day head)
        selected_records = select_records_in_period(df, start_date, end_date, 'arrive_datetime', 'depart_datetime') # get records for this periods
        
        #### calculate values
        day_calcs = calculation_function(selected_records)
        
        #### assign to dataframe
        
        daily_features.loc[start_date] = pd.Series(day_calcs) #, inplace=True)#[start_date, 'test'] = 5
    
    return(daily_features)


datetime_index = target.index
datetime_index.name = ''

EDsummary = get_daily_df(ED, datetime_index, 'ED')

In [80]:
EDsummary.to_pickle('./../../data/interim/D2_timeseries/EDsummary.pkl')